<div style="text-align: center; font-size: 30px;">
Prácticas de Estadística<br/>
</div>
<div style="text-align: center; font-size: 30px;">
Gráficos de control estadístico de la calidad
</div>
<div style="text-align: center; font-size: 16px; font-style: italic">
Material elaborado por M. Dolores Frías, Jesús Fernández y Carmen M. Sordo profesores del Departamento de Matemática Aplicada y Ciencias de la Computación de la Universidad de Cantabria.
</div>

# Objetivos

En esta práctica vamos a utilizar R para representar los gráficos de control de Shewhart. En particular nos centraremos solo en los tipos de gráficos vistos en clase: gráficos c, gráficos np y gráficos X y R.              

Consideraremos una serie de ejemplos que nos permitan hacer los cálculos
necesarios para representar las variables correspondientes en los gráficos de
control. Además veremos como seleccionar las opciones más adecuadas en las
representaciones gráficas con R de forma que se pueda concluir si el sistema
analizado está en estado de control o fuera de control.

En R existe un paquete específico de control de calidad con el que se pueden
representar los gráficos de Shewhart. Se trata del paquete *Quality
Control Charts* (*qcc*) que debemos instalar (si no lo está ya con la orden `install.packages("qcc")`) y cargar (`library(qcc)`). Recordar que la instalación solo debe hacerse una vez, mientras que la librería se debe cargar en cada sesión que abramos de R en la que queramos usar dicha librería.

En esta práctica vamos a comparar los gráficos que proporciona la librería `qcc` con los que podemos obtener cuando programamos las fórmulas vistas en clase. Vereis que es mucho más eficiente utilizar esta librería. 

In [ ]:
# Instalacion de la libreria
install.packages("qcc")
# Carga de la libreria
library(qcc)

# Gráfico c

Comenzaremos analizando en detalle como obtener la representación del gráfico c a través de un ejemplo. Recordad que este tipo de gráfico analiza el número total de defectos durante sucesivos intervalos de tiempo o espacio de longitud fija.

*Un pequeño restaurante de carretera ha recogido todos los meses durante dos años el número de quejas que ha recidido en relación a los servicios que presta. Realiza un gráfico de control e indica si el proceso se encuentra bajo control o no, utilizando los datos recogidos en el fichero quejas.xls*.

En primer lugar cargamos el fichero de datos. La función `read_excel` incluida en el paquete `readxl` nos permite leer ficheros de excel (.xls). Como ya sabemos también es posible guardar esos datos en formato *csv* desde una hoja de cálculo e importar los datos de ese fichero .csv directamente desde R con las órdenes que ya conocemos. 

In [ ]:
install.packages("readxl") # Ejecutar solo si gdata no esta instalada.
library(readxl) # Contiene la funcion read_excel

In [ ]:
# Establecemos directorio de trabajo
setwd("data/") 
# Se importan los datos
quejas <- read_excel("quejas.xls")
attach(quejas)
head(quejas) # Muestra las primeras filas del fichero

La variable a analizar en este caso es el número de quejas al mes que se reciben en el restaurante. Al tratarse de una variable de Poisson (Po($\lambda$=c)) utilizaremos un gráfico tipo c. 

In [ ]:
# Representacion de la variable
plot(Quejas, type="l", lwd=2, col="black", main="Grafico c", 
     ylab="Número de quejas", xlab="Meses")

Ya que no disponemos de un valor histórico de c, debemos calcular el límite central (CL, del inglés Central Line) y los límites superior (UCL, del inglés Upper Control Limit) e inferior de control (LCL, del inglés Lower Control Limit) con los datos de la  muestra utilizando la formulación de clase. Antes de eso debemos comprobar si la variable de Poisson que estamos analizando se puede aproximar con una distribución Normal (es decir comprobar si $\lambda=c>5$).

In [ ]:
# Condiciones de validez. c>5
c <- mean(Quejas); c 

Ya que se cumple que c>5, calculamos los límites de control. 

In [ ]:
# Limites de control 
CL <- c; CL
LCL <- CL-3*sqrt(CL); LCL
UCL <- CL+3*sqrt(CL); UCL

Como el valor de LCL que obtenemos es negativo debemos llevar este límite a 0 ya que el número de quejas no puede ser negativo. 

In [ ]:
LCL <- 0

Representamos esos límites en el gráfico con la función `abline` que nos permite trazar fácilmente rectas en un gráfico ya creado. Acceder a la ayuda de esta función para más detalles.

In [ ]:
# Representacion de la variable
plot(Quejas, type="l", lwd=2, col="black", main="Grafico c", 
     ylab="Número de quejas", xlab="Meses")
# Representacion de los limites
abline(h=CL, col="blue", lwd=2)
abline(h=c(LCL,UCL), col="red", lwd=2, lty=2)

Este es el gráfico resultante usando funciones de R para dibujar que ya conocemos. Sin embargo, de forma más sencilla y con una sola línea de código, la librería `qcc` nos permite obtener ese mismo gráfico con la orden:

In [ ]:
# Grafico c
qcc(Quejas, type="c", xlab="Meses", ylab="Número de quejas", title="Gráfico c")

Consultar la ayuda de la función *qcc* para ver por ejemplo que otro tipo de gráfico de control se puede obtener a través del argumento *type*, para ver como se podría introducir un valor histórico para c, en caso de conocerlo, o bien para ver como mejorar el gráfico añadiendo etiquetas a los ejes o título al gráfico.

También podemos guardar la información que se muestra en el gráfico anterior en un objeto de clase *qcc* y usar esos valores para otros cálculos posteriores. 

In [ ]:
# Grafico c
cplot <- qcc(Quejas, type="c", xlab="Meses", ylab="Número de quejas", title="Gráfico c")
class(cplot)
cplot$limits # Muestra los valores de LCL y UCL
cplot$violations$beyond.limits # Indica la posicion de los valores fuera de los limites.

Del gráfico obtenido observamos que hay un punto fuera de los límites de control (en el mes 14), por tanto el proceso no está en estado de control. Habría que analizar que ha dado lugar a ese valor tan alto de quejas para ese mes concreto.

Al haber utilizado los datos de la muestra para calcular los límites debemos repetir el gráfico eliminando ese punto situado por encima del límite superior con objeto de ver si aparecen otras alarmas enmascaradas.

In [ ]:
# Se eliminan las alarmas
new_Quejas <- Quejas[-cplot$violations$beyond.limits]
# Representamos el grafico de nuevo
cplot <-  qcc(new_Quejas, type="c", xlab="Meses", ylab="Número de quejas", title="Gráfico c")

Como vemos ya no se detectan más valores de la variable fuera de los límites. 

Recordar que una vez que todos los puntos están dentro de los límites superior e inferior, debemos analizar si hay otro tipo de alarmas como rachas, tendencia, superestabilidad, etc. En algunos casos veremos que el propio R destaca con puntos amarillos algunas de estas alarmas. En cualquier caso podemos añadir al último gráfico obtenido las líneas correspondientes a $CL \pm \sigma$ y $CL \pm 2\sigma$ ya que son necesarias para determinar algunas de estas alarmas. Para ello podemos ejecutar los siguientes comandos:

In [ ]:
LC <- cplot$center
sigma <- cplot$std.dev
# Limites para +-sigma y +-2sigma
warn.limits.1 <- c(LC-sigma, LC+sigma)  # calcula LC-sigma, LC+sigma
warn.limits.2 <- c(LC-2*sigma, LC+2*sigma) # calcula LC-2*sigma, LC+2*sigma
# Grafico con todos los limites
plot(cplot, restore.par=FALSE, xlab="Meses", ylab="Número de quejas", title="Gráfico c") 
abline(h=warn.limits.1, col="red", lty=2) 
abline(h=warn.limits.2, col="red", lty=2)
detach(quejas) 

La opción `restore.par` que hemos usado en la función `plot` anterior es un valor lógico que nos permite restaurar o no los parametros de un gráfico ya realizado. Si queremos añadir puntos o lineas a ese gráfico ya creado, como en este caso, establecemos este argumento como FALSE, es decir, que no cambien esos parámetros del gráfico inicial.

Observando el gráfico anterior vemos que no hay ninguna otra alarma de las mencionadas antes. 

Por último recordar que una vez terminado el estudio con un data frame, debemos *desenganchar* ese data frame con la orden `detach`. De lo contrario, puede haber confusión si cargamos una variable que tiene el mismo nombre que la columna del data frame que ya fue *adjuntado* con la orden `attach`.

# Gráfico np

Veamos ahora como representar un gráfico np. Recordemos que este tipo de gráfico se emplea para representar una variable aleatoria que cuenta la cantidad de unidades defectuosas en una muestra de tamaño n. Como sabemos esta variable sigue una distribución Binomial, B(n,p). 

Vamos a analizar este tipo de gráfico para el siguiente ejemplo.

*Una empresa fabrica pequeñas piezas de PVC mediante un proceso mecanizado. Al analizar las piezas se puede determinar si estas tienen las dimensiones adecuadas o no, en cuyo caso se considera defectuosa. La empresa quiere elaborar un gráfico de control para controlar el número de piezas defectuosas producidas por la máquina. Para ello se seleccionaron 30 muestras de tamaño 50 cuyos datos se encuentran en el fichero pvc.xls. Construya el gráfico de control para la empresa y analice la información obtenida.*

Comenzamos importando los datos del fichero:

In [ ]:
# Se importan los datos
pvc <- read_excel("pvc.xls")
attach(pvc)
head(pvc) # Muestra las primeras filas del fichero

Comprobamos si podemos suponer que la variable Binomial sigue una distribución Normal ($np>5$ y $n(1-p)>5$) a partir de los datos de la muestra ya que no disponemos del valor histórico de la probabilidad de éxito (p).

In [ ]:
n <- 50 # Tamaño de muestra
k <- nrow(pvc) # Numero de muestras analizadas
# Probabilidad de exito
p_i <- defectuosas/n
p <- mean(p_i); p
# Condiciones de validez
n*p; n*(1-p)

Como vemos las dos condiciones necesarias se cumplen en este caso por lo que procedemos a calcular los límites de control y representar el gráfico np. Lo hacemos primero usando funciones de R que ya conocemos para representar datos. 

In [ ]:
# Limites de control
CL <- mean(defectuosas); CL
LCL <- CL-3*sqrt(CL*(1-p)); LCL
UCL <- CL+3*sqrt(CL*(1-p)); UCL
# Grafico np
plot(muestra, defectuosas, type="l", lwd=2, col="black", main="Grafico np",
     ylim=c(0,25), xlab="Número de muestra", ylab="Número de piezas defectuosas")
abline(h=CL, col="blue", lwd=2)
abline(h=c(LCL,UCL), col="red", lwd=2, lty=2)

La librería `qcc` nos permite obtener ese mismo gráfico más fácilmente y con más información con la orden:

In [ ]:
npplot <- qcc(defectuosas, type = "np", sizes=n, xlab="Número de muestra",
    ylab="Número de piezas defectuosas", title="Grafico np")

Se observa que hay dos puntos por encima del límite superior de control, luego el proceso no se encuentra en estado de control. Habría que investigar que ha dado lugar a esos dos valores con un número de piezas defectuosas tan alto para poder evitar que el problema se repita en el proceso de fabricación. 

Además, dado que los límites se han calculado a partir de los datos muestrales y no de un valor histórico de p, debemos repetir el gráfico eliminando las muestras que han generado esas alarmas con el fin de comprobar si aparecen otras alarmas. Si eso ocurre habría que eliminarlas e investigar también su causa. 

Repite el gráfico elimando esas alarmas. ¿Qué observas en el nuevo gráfico? 

Recordar que, una vez que todos los puntos están dentro de los límites de control superior e inferior, debemos analizar si hay otro tipo de alarmas como rachas, tendencia, superestabilidad, etc. Realiza este análisis e indica si observas alguna alarma de este tipo. 

Una vez terminado con ese data frame debemos ejecutar la orden:

In [ ]:
detach(pvc)

# Gráficos X y R

Estos son gráficos de control que analizan dos características de tipo continuo: la media (gráfico X) y la variabilidad a través del rango (gráfico R). Analizaremos estos gráficos de control a partir del siguiente ejemplo.

*En una fábrica que produce tuberías se han medido los diámetros (en mm) de 16 tuberías. En el fichero tuberias.txt se muestran los datos obtenidos a lo largo de 8 intervalos de muestreo sucesivos, en cada uno de los cuales se han seleccionado al azar 2 tuberías para medir sus diámetros xt1 y xt2. ¿Puede decirse que la media y la variabilidad del proceso están bajo control?*

Al tratarse de la variable aleatoria del diámetro de la tubería, debemos analizar el gráfico X y el R de forma simultánea. 

Primero importamos los datos del fichero:

In [ ]:
# Importamos los datos
tuberias <- read.csv("tuberias.txt", sep="")
head(tuberias)

Calculamos ahora los datos necesarios para representar el gráfico X. En este caso como el tamaño de muestra es 2, el coeficiente $d_2$ tendrá un valor de 1.128. 

In [ ]:
# Valor del coeficiente d2 para n=2
d2 <- 1.128 
# Tamaño de la muestra
n <- ncol(tuberias)
# Calculo de la media por filas
xbar <- apply(tuberias,1,mean)
# Calculo del rango por filas
R_t <- as.vector(diff(apply(tuberias,1,range)))
# Rango medio
Rbar <- mean(R_t)
# Estimador de la desviacion tipica de la media muestral
sigma_xbar <- Rbar/(d2*sqrt(n))
# Calculo de los limites
CL  <- mean(xbar); CL
LCL <- CL-3*sigma_xbar; LCL
UCL <- CL+3*sigma_xbar; UCL

Una vez calculados los límites de control obtenemos el gráfico X de la forma: 

In [ ]:
# Grafico X
plot(xbar, type="l", lwd=2, col="black", main="Grafico X", ylab="Diámetro (mm)", 
  xlab="Número de muestra", ylim=c(75,85))
abline(h=CL, col="blue", lwd=2)
abline(h=c(LCL,UCL), col="red", lwd=2, lty=2)

Del gráfico X anterior podemos concluir que no hay alarmas debidas a valores por encima del límite superior ni por debajo del límite inferior de control.  

Hacemos ahora los cálculos necesarios para representar la variabilidad del proceso a través del gráfico R. El coeficiente que necesitamos en este caso es: $d_R=1.323$. 

In [ ]:
# Calculo de los limites
dR <- 1.323
# Estimador de la desviacion tipica del rango muestral
sigma_R <- Rbar/dR
CL <- Rbar; CL
LCL <- CL-3*sigma_R; LCL
LCL <- 0
UCL <- CL+3*sigma_R; UCL

LCL es un valor negativo que en este caso no tiene sentido puesto que la variable es una distancia, por lo que hemos establecido su valor como cero.

In [ ]:
# Grafico R
plot(R_t, type="l", lwd=2, col="black", main="Grafico R", ylab="Diámetro (mm)", 
  xlab="Número de muestra", ylim=c(0,5))
abline(h=CL, col="blue", lwd=2)
abline(h=c(LCL,UCL), col="red", lwd=2, lty=2)

Observamos que también la variabilidad del proceso está bajo control. Recordar que si en alguno de los gráficos hubiéramos obtenido algún punto fuera de los límites sería necesario, eliminar esas alarmas y repetir ambos gráficos aunque solo se hayan observado alarmas en uno de ellos. Debemos repetirlos ya que los límites de ambos han sido calculados a partir de los datos de la muestra.

La librería `qcc` nos permite obtener una versión más completa de estos dos gráficos, además de proporcionar información adicional. Para ello ejecutando los comandos:

In [ ]:
# Grafico tipo X
Xplot <- qcc(tuberias, type="xbar", xlab="Número de muestra", ylab="Diámetro (mm)", 
             title="Gráfico X")
# Grafico tipo R
Rplot <- qcc(tuberias, type="R", xlab="Número de muestra", ylab="Diámetro (mm)", 
             title="Gráfico R")

Como siempre, una vez que todos los puntos están dentro de los límites, debemos analizar si hay otro tipo de alarmas como rachas, tendencia, superestabilidad, etc.
Hacerlo y comprobar si existe alguna alarma de este tipo.

<div class="alert alert-block alert-info">
<strong>PRACTICA TÚ MISMO</strong>
 
- En una fábrica de azulejos se hornean azulejos de un metro de largo y 0.33 de ancho. Debido a la dimensión de los azulejos es normal que en algunos casos los azulejos muestren deformidades. Con el fin de controlar este problema, la fábrica realiza un control de calidad entre los azulejos fabricados en un mismo día. En el fichero *azulejos.xls* se recogen el número de deformidades contabilizadas en las muestras de azulejos analizadas en distintos días de producción. Realice un gráfico de control y determine el estado del proceso de fabricación de azulejos.
   
</n>

- La fabrica de azulejos está ahora interesada en estudiar la cantidad de unidades defectuosas en la muestra. A partir de los datos del fichero *azulejos.xls* represente el gráfico de control correspondiente y determine el estado del proceso si se sabe que cada día se selecciona una muestra de 25 azulejos para realizar el control de calidad.

</n>

- Una fábrica de pasta alimentaria ha sacado al mercado un nuevo tipo de pasta baja en grasa. El departamento de calidad de la empresa ha ido tomando muestras de la producción generada a intervalos de tiempo regulares con el fin de controlar el porcentaje de grasa por 100gr de producto.  El fichero *pasta.xls* recoge los resultados de los sucesivos análisis realizados agrupados de cinco en cinco. Utilizando estos datos, estudie si la media y la variabilidad del proceso de fabricación están bajo control.

</n>

- Una fábrica de productos mecanizados lleva la cuenta del número de piezas defectuosas que producen cada día. Los datos recogidos en el último mes son los siguientes:

  19 10 20 12 16 14 27 20 17 12 14 10 14 21  9 17 18 18 13 15 12 11 20 10 13 14 17 19 16 13

  Representar los datos en un gráfico de Shewhart adecuado y determinar si hay alarmas en el sistema. 

</div>